# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part 4: Training our End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a simple - but fairly effective - logistic regression model.  More generally, however, Snorkel plugs in with many ML libraries including [TensorFlow](https://www.tensorflow.org/), making it easy to use almost any state-of-the-art model as the end extractor!

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass, and load the test set:

In [2]:
from snorkel.models import candidate_subclass
Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

# 1 Training a `SparseLogReg` Discriminative Model
We use the training marginals to train a discriminative model that classifies each `Candidate` as a true or false mention. We'll use a random hyperparameter search, evaluated on the development set labels, to find the best hyperparameters for our model. To run a hyperparameter search, we need labels for a development set. If they aren't already available, we can manually create labels using the Viewer.

## Feature Extraction
Instead of using a deep learning approach to start, let's look at a standard sparse logistic regression model. First, we need to extract out features. This can take a while, but we only have to do it once!

In [3]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

In [5]:
F_train = featurizer.load_matrix(session, split=0)
F_dev = featurizer.load_matrix(session, split=1)
F_test = featurizer.load_matrix(session, split=2)

if F_train.size == 0:    
    %time F_train = featurizer.apply(split=0)
if F_dev.size == 0:     
    %time F_dev  = featurizer.apply_existing(split=1)
if F_test.size == 0:
    %time F_test = featurizer.apply_existing(split=2)



First, reload the training marginals:

In [6]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, F_train, split=0)

In [7]:
from snorkel.learning import SparseLogisticRegression
disc_model = SparseLogisticRegression()

The following code performs model selection by tuning our learning algorithm's hyperparamters.

In [12]:
from snorkel.learning.utils import MentionScorer
from snorkel.learning import RandomSearch, ListParameter, RangeParameter

# Searching over learning rate
rate_param = RangeParameter('lr', 1e-6, 1e-2, step=1, log_base=10)
l1_param  = RangeParameter('l1_penalty', 1e-6, 1e-2, step=1, log_base=10)
l2_param  = RangeParameter('l2_penalty', 1e-6, 1e-2, step=1, log_base=10)

searcher = RandomSearch(session, disc_model, 
                        F_train, train_marginals, 
                        [rate_param, l1_param, l2_param], 
                        n=4)

Initialized RandomSearch search of size 4. Search space size = 125.


In [13]:
from snorkel.annotations import load_gold_labels

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev.shape

(228, 1)

In [14]:
np.random.seed(1701)
searcher.fit(F_dev, L_gold_dev, n_epochs=400, rebalance=0.5, print_freq=25)

[1] Testing lr = 1.00e-02, l1_penalty = 1.00e-02, l2_penalty = 1.00e-04
[SparseLR] lr=0.01 l1=0.01 l2=0.0001
[SparseLR] Building model
[SparseLR] Training model
[SparseLR] #examples=392  #epochs=400  batch size=100
[SparseLR] Epoch 0 (0.47s)	Avg. loss=1.668130	NNZ=136398
[SparseLR] Epoch 25 (4.36s)	Avg. loss=0.599224	NNZ=136398
[SparseLR] Epoch 50 (8.61s)	Avg. loss=0.586881	NNZ=136398
[SparseLR] Epoch 75 (12.86s)	Avg. loss=0.585800	NNZ=136398
[SparseLR] Epoch 100 (16.98s)	Avg. loss=0.584735	NNZ=136398
[SparseLR] Epoch 125 (20.99s)	Avg. loss=0.584487	NNZ=136398
[SparseLR] Epoch 150 (24.94s)	Avg. loss=0.584277	NNZ=136398
[SparseLR] Epoch 175 (28.87s)	Avg. loss=0.584588	NNZ=136398
[SparseLR] Epoch 200 (32.84s)	Avg. loss=0.583887	NNZ=136398
[SparseLR] Epoch 225 (37.00s)	Avg. loss=0.584278	NNZ=136398
[SparseLR] Epoch 250 (41.31s)	Avg. loss=0.584071	NNZ=136398
[SparseLR] Epoch 275 (45.97s)	Avg. loss=0.584025	NNZ=136398
[SparseLR] Epoch 300 (50.39s)	Avg. loss=0.583645	NNZ=136398
[SparseLR] Ep

,lr,l1_penalty,l2_penalty,Prec.,Rec.,F1
3,0.0010,0.010000,0.001000,0.800000,0.571429,0.666667
0,0.0100,0.010000,0.000100,0.555556,0.714286,0.625000
1,0.0001,0.000001,0.000001,0.000000,0.000000,0.000000
2,0.0010,0.000001,0.000100,0.000000,0.000000,0.000000


## Examining Features
Extracting features allows us to inspect and interperet our learned weights 

In [17]:
w, _ = disc_model.get_weights()
largest_idxs = reversed(np.argsort(np.abs(w))[-50
                                              :])
for i in largest_idxs:
    print('Feature: {0: <70}Weight: {1:.6f}'.format(F_train.get_key(session, i).name, w[i]))

Feature: TDL_INV_LEMMA:SEQ-BETWEEN[husband]                                    Weight: 0.882935
Feature: TDL_LEMMA:LEFT-OF-MENTION[wife]                                       Weight: 0.677696
Feature: TDL_LEMMA:SEQ-BETWEEN[husband]                                        Weight: 0.635757
Feature: TDL_INV_LEMMA:SEQ-BETWEEN[wife]                                       Weight: 0.598728
Feature: TDL_LEMMA:SEQ-BETWEEN[wife]                                           Weight: 0.536108
Feature: TDL_INV_LEMMA:SEQ-BETWEEN[, he]                                       Weight: 0.487989
Feature: TDL_LEMMA:SEQ-BETWEEN[she husband]                                    Weight: 0.478656
Feature: TDL_LEMMA:LEFT-OF-MENTION[face-to-face with]                          Weight: 0.467664
Feature: TDL_LEMMA:LEFT-OF-MENTION[or]                                         Weight: 0.467325
Feature: TDL_LEMMA:RIGHT-OF-MENTION[, be]                                      Weight: 0.449287
Feature: TDL_DEP_LABEL:BETWEEN-MENTION-a

## Evaluate on Test Data

In [18]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

In [19]:
_, _, _, _ = disc_model.score(session, F_test, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.667
Neg. class accuracy: 0.993
Precision            0.667
Recall               0.667
F1                   0.667
----------------------------------------
TP: 4 | FP: 2 | TN: 270 | FN: 2



# 2: Training an LSTM Discriminative Model
Deep learning allows us to train models without manually definining features

In [20]:
train = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

print('Training set:\t{0} candidates'.format(len(train)))
print('Dev set:\t{0} candidates'.format(len(dev)))
print('Test set:\t{0} candidates'.format(len(test)))

Training set:	4781 candidates
Dev set:	228 candidates
Test set:	278 candidates


Load dev labels and convert to [0, 1] range

In [21]:
dev_labels = (np.ravel(L_gold_dev.todense()) + 1) / 2

In [26]:
from snorkel.contrib.rnn import reRNN

train_kwargs = {
    'lr':         0.01,
    'dim':        100,
    'n_epochs':   50,
    'dropout':    0.5,
    'rebalance':  0.25,
    'print_freq': 5,
    'batch_size': 32
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train, train_marginals, dev_candidates=dev,
           dev_labels=dev_labels, **train_kwargs)

[reRNN] Dimension=100  LR=0.01
[reRNN] Begin preprocessing
[reRNN] Loaded 228 candidates for evaluation
[reRNN] Preprocessing done (2.15s)
[reRNN] Training model
[reRNN] #examples=784  #epochs=50  batch size=32
[reRNN] Epoch 0 (17.88s)	Average loss=0.577117	Dev F1=0.00
[reRNN] Epoch 5 (104.94s)	Average loss=0.494814	Dev F1=0.00
[reRNN] Epoch 10 (207.12s)	Average loss=0.493529	Dev F1=13.33
[reRNN] Epoch 15 (304.65s)	Average loss=0.493353	Dev F1=20.00
[reRNN] Epoch 20 (400.57s)	Average loss=0.492647	Dev F1=0.00
[reRNN] Epoch 25 (495.43s)	Average loss=0.492260	Dev F1=0.00
[reRNN] Epoch 30 (590.52s)	Average loss=0.492343	Dev F1=0.00
[reRNN] Epoch 35 (685.49s)	Average loss=0.493182	Dev F1=40.00
[reRNN] Epoch 40 (779.16s)	Average loss=0.492220	Dev F1=25.00
[reRNN] Epoch 45 (872.93s)	Average loss=0.491935	Dev F1=0.00
[reRNN] Epoch 49 (949.82s)	Average loss=0.491920	Dev F1=0.00
[reRNN] Training done (950.67s)


## 3. Evaluating on the Test Set

In this last section of the tutorial, we'll get the score we've been after: the performance of the extraction model on the blind test set (`split` 2). First, we load the test set labels and gold candidates we made in Part III.

In [27]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

Now, we score using the discriminative model:

In [28]:
_, _, _, _  = lstm.score(session, test, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.0
Neg. class accuracy: 1.0
Precision            0.0
Recall               0.0
F1                   0.0
----------------------------------------
TP: 0 | FP: 0 | TN: 272 | FN: 6

